# SPARQL 

<a href="https://colab.research.google.com/github/joerg84/Graph_Powered_ML_Workshop/blob/master/Sparql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip3 install rdflib

In [ ]:
import rdflib
from rdflib.namespace import DC, RDF, FOAF
from rdflib import URIRef, BNode, Literal

In [ ]:
bob = URIRef("http://example.org/people/Bob")
linda = BNode() # a GUID is generated

name = Literal('Bob') # passing a string
age = Literal(24) # passing a python int
height = Literal(76.5) # passing a python float

g = rdflib.Graph()

g.add( (bob, RDF.type, FOAF.Person) )
g.add( (bob, FOAF.name, name) )
g.add( (bob, FOAF.knows, linda) )
g.add( (linda, RDF.type, FOAF.Person) )
g.add( (linda, FOAF.name, Literal('Linda') ) )

#print all triples
for s, p, o in g:
   print((s, p, o))

Print all triples from Sparql

In [ ]:
result = g.query(
    """SELECT *
  WHERE
  {?s ?p ?o}
""")
for row in result:
    print(row)


In [ ]:
result = g.query(
    """SELECT DISTINCT ?aname ?bname
       WHERE {
          ?a foaf:knows ?b .
          ?a foaf:name ?aname .
          ?b foaf:name ?bname .
       }""", initNs={ 'foaf': FOAF })

for row in result:
    print("%s knows %s" % row)

Slightly larger Graph:

In [ ]:
g = rdflib.Graph()
g.parse("http://www.w3.org/People/Berners-Lee/card")

print("Graph has %s statements." % len(g))

for s, p, o in g:
   print((s, p, o))